# STEP 1: Camera Calibration

In [1]:
import numpy as np
import cv2
import glob

cv2.ocl.setUseOpenCL(False)

CHECKERBOARD = (8, 6)  # internal corners
objpoints = []
imgpoints = []

objp = np.zeros((CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)

# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)


images = glob.glob("/Users/macbookpro/JupyterLab/computer_vision/module2/step1_images/*.JPG")
if len(images) == 0:
    raise RuntimeError("No calibration images found.")

for fname in images:
    img = cv2.imread(fname)
    if img is None:
        continue

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    ret, corners = cv2.findChessboardCornersSB(gray, CHECKERBOARD)
    print(fname, "→", ret)

    if ret:
        corners_refined = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria=criteria)

        objpoints.append(objp.copy())
        imgpoints.append(corners_refined)

        cv2.drawChessboardCorners(img, CHECKERBOARD, corners_refined, ret)
        cv2.imshow("Corners", img)
        cv2.waitKey(500)

cv2.destroyAllWindows()

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

print("RMS reprojection error:", ret)
print("Camera Matrix:\n", mtx)
print("Distortion Coefficients:\n", dist)


[ERROR:1@0.228] global ocl.cpp:4679 createFromBinary OpenCL error CL_INVALID_VALUE (-30) during call: clCreateProgramWithBinary
[ERROR:2@0.356] global ocl.cpp:4679 createFromBinary OpenCL error CL_INVALID_VALUE (-30) during call: clCreateProgramWithBinary
[ERROR:1@0.357] global ocl.cpp:3769 set OpenCL: Kernel(boxFilter)::set(arg_index=0, flags=18): can't create cl_mem handle for passed UMat buffer (addr=0x308c46af0)
[ERROR:1@0.369] global ocl.cpp:3769 set OpenCL: Kernel(warpAffine)::set(arg_index=0, flags=2): can't create cl_mem handle for passed UMat buffer (addr=0x308c47150)


/Users/macbookpro/JupyterLab/computer_vision/module2/step1_images/IMG_1316.JPG → True
/Users/macbookpro/JupyterLab/computer_vision/module2/step1_images/IMG_1315.JPG → True
/Users/macbookpro/JupyterLab/computer_vision/module2/step1_images/IMG_1311.JPG → True
/Users/macbookpro/JupyterLab/computer_vision/module2/step1_images/IMG_1312.JPG → True
RMS reprojection error: 1.5887852622561587
Camera Matrix:
 [[3.62283510e+03 0.00000000e+00 1.59359289e+03]
 [0.00000000e+00 3.62721214e+03 1.90103796e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Distortion Coefficients:
 [[ 2.50939173e-01 -2.65946131e+00 -1.66446788e-03  2.32116375e-03
   9.17236600e+00]]


# STEP 2: Implementation for Perspective Projection

In [2]:
def find_real_world_dimensions(image_path, mtx, dist, object_distance_mm):
    """
    Find real-world 2D dimensions of an object using perspective projection.
    
    Perspective projection equations:
    X_real = (pixel_distance_x * Z) / fx
    Y_real = (pixel_distance_y * Z) / fy
    
    Args:
        image_path: path to image with object to measure
        mtx: camera matrix from calibration
        dist: distortion coefficients from calibration
        object_distance_mm: distance from camera to object in mm
    
    Returns:
        Dictionary with real-world dimensions
    """
    # Load image
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Cannot load image: {image_path}")
    
    h, w = img.shape[:2]
    
    # Undistort image using calibration parameters
    newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))
    img_undistorted = cv2.undistort(img, mtx, dist, None, newcameramtx)
    
    # Extract camera parameters
    fx = mtx[0, 0]  # focal length x
    fy = mtx[1, 1]  # focal length y
    cx = mtx[0, 2]  # principal point x
    cy = mtx[1, 2]  # principal point y
    
    print(f"\n{'='*60}")
    print(f"Camera Parameters:")
    print(f"{'='*60}")
    print(f"Focal Length (fx): {fx:.2f} pixels")
    print(f"Focal Length (fy): {fy:.2f} pixels")
    print(f"Principal Point (cx, cy): ({cx:.2f}, {cy:.2f}) pixels")
    print(f"Object Distance: {object_distance_mm:.2f} mm ({object_distance_mm/304.8:.2f} ft)")
    print(f"{'='*60}\n")
    
    # Interactive point selection
    points = []
    img_display = img_undistorted.copy()
    
    def click_event(event, x, y, flags, params):
        if event == cv2.EVENT_LBUTTONDOWN:
            points.append((x, y))
            cv2.circle(img_display, (x, y), 8, (0, 255, 0), -1)
            cv2.putText(img_display, f"P{len(points)}: ({x},{y})", (x+15, y-10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
            
            if len(points) == 2:
                # Draw line between points
                cv2.line(img_display, points[0], points[1], (0, 255, 0), 3)
                
            cv2.imshow('Select Object Points', img_display)
            
            if len(points) == 2:
                print(f"Point 1: {points[0]}")
                print(f"Point 2: {points[1]}")
                print("\nPress any key to calculate dimensions...")
    
    cv2.imshow('Select Object Points', img_display)
    cv2.setMouseCallback('Select Object Points', click_event)
    
    print("="*60)
    print("INSTRUCTIONS:")
    print("="*60)
    print("1. Click TWO points on the object to measure")
    print("   (e.g., opposite corners of a rectangle)")
    print("2. Press any key after selecting both points")
    print("3. Press 'ESC' to cancel")
    print("="*60)
    
    # Wait for user to select points and press a key
    key = cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    if key == 27:  # ESC key
        print("Measurement cancelled.")
        return None
    
    if len(points) < 2:
        raise ValueError("Need exactly 2 points to measure")
    
    # Calculate pixel dimensions
    x1, y1 = points[0]
    x2, y2 = points[1]
    
    pixel_width = abs(x2 - x1)
    pixel_height = abs(y2 - y1)
    pixel_diagonal = np.sqrt(pixel_width**2 + pixel_height**2)
    
    print(f"\n{'='*60}")
    print(f"PIXEL MEASUREMENTS:")
    print(f"{'='*60}")
    print(f"Width:    {pixel_width:.2f} pixels")
    print(f"Height:   {pixel_height:.2f} pixels")
    print(f"Diagonal: {pixel_diagonal:.2f} pixels")
    
    # Apply perspective projection equations
    # Real_dimension = (pixel_dimension * distance) / focal_length
    real_width_mm = (pixel_width * object_distance_mm) / fx
    real_height_mm = (pixel_height * object_distance_mm) / fy
    real_diagonal_mm = np.sqrt(real_width_mm**2 + real_height_mm**2)
    
    # Convert to different units
    real_width_cm = real_width_mm / 10
    real_height_cm = real_height_mm / 10
    real_diagonal_cm = real_diagonal_mm / 10
    
    real_width_inches = real_width_mm / 25.4
    real_height_inches = real_height_mm / 25.4
    real_diagonal_inches = real_diagonal_mm / 25.4
    
    real_width_ft = real_width_mm / 304.8
    real_height_ft = real_height_mm / 304.8
    
    print(f"\n{'='*60}")
    print(f"REAL-WORLD DIMENSIONS:")
    print(f"{'='*60}")
    print(f"Width:")
    print(f"  {real_width_mm:.2f} mm")
    print(f"  {real_width_cm:.2f} cm")
    print(f"  {real_width_inches:.2f} inches")
    print(f"  {real_width_ft:.4f} ft")
    print(f"\nHeight:")
    print(f"  {real_height_mm:.2f} mm")
    print(f"  {real_height_cm:.2f} cm")
    print(f"  {real_height_inches:.2f} inches")
    print(f"  {real_height_ft:.4f} ft")
    print(f"\nDiagonal:")
    print(f"  {real_diagonal_mm:.2f} mm")
    print(f"  {real_diagonal_cm:.2f} cm")
    print(f"  {real_diagonal_inches:.2f} inches")
    print(f"{'='*60}\n")
    
    # Display result image with measurements
    img_result = img_undistorted.copy()
    cv2.circle(img_result, points[0], 8, (0, 255, 0), -1)
    cv2.circle(img_result, points[1], 8, (0, 255, 0), -1)
    cv2.line(img_result, points[0], points[1], (0, 255, 0), 3)
    
    # Add text with measurements
    mid_point = ((x1 + x2) // 2, (y1 + y2) // 2 - 20)
    text = f"{real_width_inches:.1f}\" x {real_height_inches:.1f}\""
    cv2.putText(img_result, text, mid_point, 
                cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 0, 0), 3)
    
    cv2.imshow('Measurement Result', img_result)
    print("Press any key to close the result window...")
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    return {
        'width_mm': real_width_mm,
        'height_mm': real_height_mm,
        'diagonal_mm': real_diagonal_mm,
        'width_cm': real_width_cm,
        'height_cm': real_height_cm,
        'width_inches': real_width_inches,
        'height_inches': real_height_inches,
        'width_ft': real_width_ft,
        'height_ft': real_height_ft,
        'pixel_width': pixel_width,
        'pixel_height': pixel_height,
        'distance_mm': object_distance_mm,
        'distance_ft': object_distance_mm / 304.8
    }

# STEP 3: Validation Experiment

In [4]:
# Distance from camera to object: 9.0 feet
object_distance_mm = 9.0 * 304.8  # Convert to mm

# Replace with your actual object image path
image_path = "/Users/macbookpro/JupyterLab/computer_vision/module2/step3_images/IMG_1323.JPG"

# Measure the object
results = find_real_world_dimensions(image_path, mtx, dist, object_distance_mm)


Camera Parameters:
Focal Length (fx): 3622.84 pixels
Focal Length (fy): 3627.21 pixels
Principal Point (cx, cy): (1593.59, 1901.04) pixels
Object Distance: 2743.20 mm (9.00 ft)

INSTRUCTIONS:
1. Click TWO points on the object to measure
   (e.g., opposite corners of a rectangle)
2. Press any key after selecting both points
3. Press 'ESC' to cancel
Point 1: (1297, 1883)
Point 2: (1729, 2190)

Press any key to calculate dimensions...

PIXEL MEASUREMENTS:
Width:    432.00 pixels
Height:   307.00 pixels
Diagonal: 529.97 pixels

REAL-WORLD DIMENSIONS:
Width:
  327.11 mm
  32.71 cm
  12.88 inches
  1.0732 ft

Height:
  232.18 mm
  23.22 cm
  9.14 inches
  0.7617 ft

Diagonal:
  401.13 mm
  40.11 cm
  15.79 inches

Press any key to close the result window...


In [5]:
import numpy as np

def calculate_measurement_error(measured_width_inch, measured_height_inch, 
                               true_width_inch, true_height_inch, 
                               print_results=True):
    """
    Calculate measurement errors between measured and true dimensions.
    
    Args:
        measured_width_inch: Width calculated by your program (inches)
        measured_height_inch: Height calculated by your program (inches)
        true_width_inch: Actual width measured with ruler/caliper (inches)
        true_height_inch: Actual height measured with ruler/caliper (inches)
        print_results: Whether to print formatted results (default: True)
    
    Returns:
        Dictionary containing all error metrics
    """
    
    # Absolute errors
    width_error_inch = measured_width_inch - true_width_inch
    height_error_inch = measured_height_inch - true_height_inch
    
    # Percentage errors
    width_error_percent = (width_error_inch / true_width_inch) * 100
    height_error_percent = (height_error_inch / true_height_inch) * 100
    
    # Average absolute percentage error
    avg_error_percent = (abs(width_error_percent) + abs(height_error_percent)) / 2
    
    # Root Mean Square Error (RMSE)
    rmse = np.sqrt((width_error_inch**2 + height_error_inch**2) / 2)
    
    # Relative RMSE (normalized by true dimensions)
    true_diagonal = np.sqrt(true_width_inch**2 + true_height_inch**2)
    relative_rmse = (rmse / true_diagonal) * 100
    
    # Mean Absolute Error (MAE)
    mae = (abs(width_error_inch) + abs(height_error_inch)) / 2
    
    # Calculate measured vs true diagonal
    measured_diagonal = np.sqrt(measured_width_inch**2 + measured_height_inch**2)
    diagonal_error = measured_diagonal - true_diagonal
    diagonal_error_percent = (diagonal_error / true_diagonal) * 100
    
    # Prepare results dictionary
    results = {
        # Measured values
        'measured_width_inch': measured_width_inch,
        'measured_height_inch': measured_height_inch,
        'measured_diagonal_inch': measured_diagonal,
        
        # True values
        'true_width_inch': true_width_inch,
        'true_height_inch': true_height_inch,
        'true_diagonal_inch': true_diagonal,
        
        # Absolute errors
        'width_error_inch': width_error_inch,
        'height_error_inch': height_error_inch,
        'diagonal_error_inch': diagonal_error,
        
        # Percentage errors
        'width_error_percent': width_error_percent,
        'height_error_percent': height_error_percent,
        'diagonal_error_percent': diagonal_error_percent,
        'avg_error_percent': avg_error_percent,
        
        # Statistical metrics
        'mae': mae,
        'rmse': rmse,
        'relative_rmse_percent': relative_rmse
    }
    
    # Print formatted results if requested
    if print_results:
        print(f"\n{'='*70}")
        print(f"MEASUREMENT ERROR ANALYSIS")
        print(f"{'='*70}")
        
        print(f"\n{'DIMENSION':<20} {'MEASURED':<15} {'TRUE':<15} {'ERROR':<15} {'ERROR %':<10}")
        print(f"{'-'*70}")
        print(f"{'Width (inches)':<20} {measured_width_inch:<15.3f} {true_width_inch:<15.3f} {width_error_inch:<+15.3f} {width_error_percent:<+10.2f}%")
        print(f"{'Height (inches)':<20} {measured_height_inch:<15.3f} {true_height_inch:<15.3f} {height_error_inch:<+15.3f} {height_error_percent:<+10.2f}%")
        print(f"{'Diagonal (inches)':<20} {measured_diagonal:<15.3f} {true_diagonal:<15.3f} {diagonal_error:<+15.3f} {diagonal_error_percent:<+10.2f}%")
        
        print(f"\n{'='*70}")
        print(f"OVERALL ERROR METRICS")
        print(f"{'='*70}")
        print(f"Average Absolute Error:     {avg_error_percent:.2f}%")
        print(f"Mean Absolute Error (MAE):  {mae:.3f} inches")
        print(f"Root Mean Square Error:     {rmse:.3f} inches")
        print(f"Relative RMSE:              {relative_rmse:.2f}%")
        
        print(f"\n{'='*70}")
        print(f"ACCURACY ASSESSMENT")
        print(f"{'='*70}")
        
        if avg_error_percent < 1:
            assessment = "EXCELLENT"
            symbol = "✓✓✓"
        elif avg_error_percent < 2:
            assessment = "VERY GOOD"
            symbol = "✓✓"
        elif avg_error_percent < 5:
            assessment = "GOOD"
            symbol = "✓"
        elif avg_error_percent < 10:
            assessment = "ACCEPTABLE"
            symbol = "⚠"
        else:
            assessment = "POOR"
            symbol = "✗"
        
        print(f"{symbol} {assessment}")
        print(f"   Average Error: {avg_error_percent:.2f}%")
        
        if avg_error_percent >= 5:
            print(f"\n{symbol} Potential issues to check:")
            print(f"   • Camera calibration quality")
            print(f"   • Distance measurement accuracy")
            print(f"   • Camera-to-object alignment (perpendicular?)")
            print(f"   • Point selection accuracy")
        
        print(f"{'='*70}\n")
    
    return results

In [7]:
result = calculate_measurement_error(
    measured_width_inch=12.9,    
    measured_height_inch=9.1,
    true_width_inch=14.01,     
    true_height_inch=9.77, 
    print_results=True
)


MEASUREMENT ERROR ANALYSIS

DIMENSION            MEASURED        TRUE            ERROR           ERROR %   
----------------------------------------------------------------------
Width (inches)       12.900          14.010          -1.110          -7.92     %
Height (inches)      9.100           9.770           -0.670          -6.86     %
Diagonal (inches)    15.787          17.080          -1.293          -7.57     %

OVERALL ERROR METRICS
Average Absolute Error:     7.39%
Mean Absolute Error (MAE):  0.890 inches
Root Mean Square Error:     0.917 inches
Relative RMSE:              5.37%

ACCURACY ASSESSMENT
⚠ ACCEPTABLE
   Average Error: 7.39%

⚠ Potential issues to check:
   • Camera calibration quality
   • Distance measurement accuracy
   • Camera-to-object alignment (perpendicular?)
   • Point selection accuracy

